In [2]:
import zipfile
import pandas as pd

In [5]:
df_ca = pd.read_pickle("data/ca_cleaned.pkl", compression= "gzip")
df_tx = pd.read_pickle("data/ca_cleaned.pkl", compression = "gzip")

In [6]:
df_ca.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
0,2009-07-01,2009-07-01,Stanislaus County,other,male,False,no_search,unknown
1,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,False,no_search,summons
2,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,True,other,summons
3,2009-07-01,2009-07-01,Stanislaus County,white,female,False,no_search,summons
4,2009-07-01,2009-07-01,Stanislaus County,hispanic,male,True,other,summons


In [11]:
df_ca["date"] = pd.to_datetime(df_ca["date"])

In [12]:
df_ca["year"] = df_ca["date"].dt.year

In [25]:
count_year_ethnicity = df_ca.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity.rename({"date": "count"}, inplace=True)

count_year_ethnicity.head(40)


year  county_name          subject_race          
2009  Alameda County       asian/pacific islander    13886
                           black                     20976
                           hispanic                  25689
                           other                     13071
                           white                     40816
      Alpine County        asian/pacific islander      341
                           black                       125
                           hispanic                    661
                           other                       348
                           white                      5630
      Amador County        asian/pacific islander      377
                           black                       184
                           hispanic                    637
                           other                       282
                           white                      5532
      Butte County         asian/pacific islander      479
      

In [26]:
count_year_gender = df_ca.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender.rename({"date": "count"}, inplace=True)

count_year_gender.head(40)

year  county_name          subject_sex
2009  Alameda County       female          34786
                           male            79651
      Alpine County        female           1895
                           male             5210
      Amador County        female           2239
                           male             4773
      Butte County         female           6795
                           male            11053
      Calaveras County     female           3282
                           male             6266
      Colusa County        female           2400
                           male             5349
      Contra Costa County  female           9552
                           male            18016
      Del Norte County     female           1861
                           male             3591
      El Dorado County     female           4201
                           male             8472
      Frenso County        female           5197
                           mal

In [27]:
df_ca.outcome.unique()

array(['unknown', 'summons', 'warning', 'citation', 'arrest'],
      dtype=object)

In [29]:
count_year_ethnicity.to_pickle("data/count_year_ethnicity.pkl", compression="gzip")
count_year_gender.to_pickle("data/count_year_gender.pkl", compression="gzip")


In [30]:
test = pd.read_pickle("data/count_year_gender.pkl", compression="gzip")

In [ ]:
sub_sample = pd.DataFrame(a).reset_index().query('year == 2009')
sub_sample = sub_sample.query("subject_sex == 'female'")
sub_sample.head()

In [ ]:
sub_sample.to_csv('arrest_2009_female_ca.csv')